In [246]:
import json
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [247]:
# Del punto dos
def process_text(documents):
    """
    Limpia y procesa los datos de texto crudo.

    Parámetros:
    - documents (dict): Un diccionario de IDs de documentos y texto crudo.

    Devuelve:
    - documents_dict (dict): Un diccionario de IDs de documentos y tokens procesados.
    """
    documents_dict = {}

    for key, value in documents.items():
        # Convierte a minúsculas
        text = value.lower()
        
        # Tokeniza en palabras
        tokens = word_tokenize(text)
        
        # Elimina tokens no alfabéticos, como puntos y comas
        tokens = [token for token in tokens if token.isalpha()]
        
        # Elimina palabras de parada
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        
        # Aplica stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
        
        documents_dict[key] = tokens

    return documents_dict

def crear_tf_idf_matrix(inverted_index: dict[dict[int]], 
                        tf_log_scale: bool = True,
                        normalize_matrix: bool = True) -> tuple[np.array, list[str], list[int]]:
    """
    Calcula la matriz TF-IDF a partir de un índice invertido.

    Esta función toma un índice invertido que mapea términos a documentos y sus frecuencias, 
    y calcula la matriz TF-IDF para cada término en cada documento. La opción de usar escala 
    logarítmica para la frecuencia de términos (TF) es configurable, así como la opción de 
    normalizar la matriz resultante.

    Args:
        inverted_index (dict[dict[int]]): 
            Un diccionario donde las claves son términos (str) y los valores son diccionarios.
            Estos diccionarios internos mapean el ID del documento (int) a la frecuencia de ese 
            término en el documento.

        tf_log_scale (bool, opcional): 
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. Si 
            se establece en False, se usará la frecuencia sin escala logarítmica.

        normalize_matrix (bool, opcional): 
            Indica si se debe normalizar la matriz TF-IDF resultante. La normalización se realiza por 
            filas, lo que significa que cada vector de documento se ajusta para que su norma sea 1. 
            Por defecto es True.

    Returns:
        tuple[np.array, list[str], list[int]]: 
            Retorna una tupla que contiene:
            - tf_idf_matrix (np.array): Un array bidimensional donde cada fila representa un documento 
              y cada columna representa un término. Los valores en la matriz son los pesos TF-IDF 
              correspondientes.
            - terms (list[str]): Una lista ordenada de los términos presentes en el índice invertido.
            - docs (list[int]): Una lista ordenada de los IDs de documentos únicos en el corpus.
    """

    # Obtiene y ordena los términos únicos en el índice invertido.
    terms = sorted(list(inverted_index.keys()))

    # Extrae y ordena los IDs de documentos únicos en el corpus.
    docs = {doc for docs_freq in inverted_index.values() for doc in docs_freq}
    docs = sorted(list(docs))

    # Calcula el Inverse Document Frequency (IDF) para cada término.
    # IDF = log10(N / df), donde N es el número total de documentos y df es la frecuencia de documentos 
    # que contienen el término.
    idf = {term: np.log10(len(docs) / len(inverted_index[term]))
           for term in terms}

    # Calcula el Term Frequency (TF) para cada término en cada documento.
    # Si se aplica escala logarítmica, se usa la fórmula: TF = log10(1 + frecuencia).
    # Si no, se utiliza la frecuencia directa.
    if tf_log_scale:
        tf = {doc: {term: np.log10(1 + inverted_index[term].get(doc, 0)) 
                    for term in inverted_index.keys()} for doc in docs}
    else:
        tf = {doc: {term: inverted_index[term].get(
            doc, 0) for term in inverted_index.keys()} for doc in docs}

    # Calcula el TF-IDF para cada término en cada documento.
    # TF-IDF = TF * IDF para cada término en cada documento.
    tf_idf = {doc: {term: tf[doc][term] * idf[term]
                    for term in terms} for doc in docs}

    # Convierte el diccionario de TF-IDF en una matriz numpy para facilitar el procesamiento posterior.
    # Cada fila de la matriz representa un documento y cada columna un término.
    tf_idf_matrix = np.array([[tf_idf[doc][term] for term in terms] for doc in docs])
    
    # Normaliza la matriz TF-IDF, de manera que la norma de cada vector de documento sea 1.
    if normalize_matrix:
        tf_idf_matrix = tf_idf_matrix / np.linalg.norm(tf_idf_matrix, axis=1, keepdims=True)
    
    # Retorna la matriz TF-IDF, la lista de términos y la lista de documentos en el orden correspondiente.
    return tf_idf_matrix, terms, docs, idf


def cosine_simi(v1: np.array, v2: np.array, asume_norm_1: bool = False) -> float:
    """
    Calcula la similitud coseno entre dos vectores.

    Esta función calcula la similitud coseno entre dos vectores `v1` y `v2`. 
    La similitud coseno es una medida de la similitud entre dos vectores 
    en un espacio vectorial que mide el coseno del ángulo entre ellos. 
    Se utiliza comúnmente en análisis de texto y en tareas de recuperación de información.

    Args:
        v1 (np.array): 
            El primer vector (como un array de NumPy) con el que se calculará la similitud.
        
        v2 (np.array): 
            El segundo vector (como un array de NumPy) con el que se calculará la similitud.
        
        asume_norm_1 (bool, opcional): 
            Si se establece en True, la función asume que ambos vectores `v1` y `v2` 
            ya están normalizados (es decir, su norma es 1). Esto permite omitir el 
            cálculo de las normas, mejorando la eficiencia. Por defecto es False.

    Returns:
        float: 
            Un valor de tipo float que representa la similitud coseno entre los dos vectores.
            Un valor cercano a 1 indica que los vectores son muy similares (paralelos), 
            mientras que un valor cercano a 0 indica que son ortogonales (no relacionados).

    """
    
    if asume_norm_1:
        # Si se asume que los vectores ya están normalizados (norma = 1), 
        # la similitud coseno se reduce al producto punto entre ellos.
        return np.dot(v1, v2)
    else:
        # Si los vectores no están normalizados, se calcula la similitud coseno 
        # como el producto punto dividido por el producto de las normas de los vectores.
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [248]:
inverted_index = json.loads(open('./output/inverted_index.json').read())

In [256]:
tf_idf_matrix, terms, docs, corpus_idf = crear_tf_idf_matrix(inverted_index, 
                                                             tf_log_scale=True,
                                                             normalize_matrix=True
                                                             )

In [260]:
terms == list(corpus_idf.keys())

True

In [261]:
text = 'hey you! i am a physician physician physician'

def crear_vector_tf_idf(text:str,
                        tf_log_scale: bool = True,
                        normalize_vector: bool = True)-> np.array:
    
    text_cln = process_text({'0': text})['0']

    if tf_log_scale:
        txt_vector = np.array([
            np.log10(1 + text_cln.count(term)) * corpus_idf[term] if term in text_cln else 0
            for term in terms])
    else:
        txt_vector = np.array([
            text_cln.count(term) * corpus_idf[term] if term in text_cln else 0
            for term in terms])

    if normalize_vector:
        txt_vector = txt_vector / np.linalg.norm(txt_vector)
        
    return txt_vector

In [262]:
crear_vector_tf_idf(text)

array([0., 0., 0., ..., 0., 0., 0.])